In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
def unwrap_trajectory(particle_positions, box_sizes):
    # Compute displacement between frames
    delta_x = np.diff(particle_positions, axis=0)
    
    # Get nearest image displacement
    delta_x = np.fix(2 * delta_x / box_sizes) * box_sizes - delta_x
    
    # Take cumulative sum of displacements
    unwrapped_positions = np.cumsum(delta_x, axis=0)
    
    return unwrapped_positions

def compute_mean_displacement(unwrapped_positions):
    # Average trajectories over all particles
    mean_displacement = np.mean(unwrapped_positions, axis=1)
    
    return mean_displacement

def compute_velocity(mean_displacement, time_interval):
    # Fit a straight line to the displacement to compute velocity
    velocity = np.polyfit(np.arange(len(mean_displacement)), mean_displacement, 1)[0]
    
    # Convert displacement to velocity (assuming time_interval is 1)
    velocity /= time_interval
    
    return velocity

In [3]:
if __name__ == "__main__":
    directory = "."  # Directory containing the data files
    time_interval = 1
    
    results = []  # List to store results
    
    for filename in os.listdir(directory):
        if filename.startswith("sed_N8000_phi") and filename.endswith("_FD.txt"):
            print(f"Processing file: {filename}")  # Print filename before loading data
            
            # Extract phi from filename
            phi = float(filename.split("_")[2].replace("phi", ""))
            
            tag = 0

            with open(os.path.join(directory, filename), 'r') as file:
                
                # Initialize arrays to store particle positions for each frame
                particle_positions = []
                box_sizes = None

                # Read the file line by line
                frame_positions = []
                row_count = 0
                for line in file:
                    if row_count % 8003 == 0:
                        N = int(line)
                    elif row_count % 8003 == 1:
                        box_sizes = np.array([float(x) for x in line.split()])
                    elif line.strip():
                        frame_positions.append([float(x) for x in line.split()])
                    else:
                        # Process the completed frame
                        particle_positions.append(np.array(frame_positions))
                        frame_positions = []
                    row_count += 1

                # Convert particle positions to numpy array
                particle_positions = np.array(particle_positions)
            
                # Initialize arrays to store mean displacements and velocities for each frame
                mean_displacements = []
                velocities = []
            
                unwrapped_positions = unwrap_trajectory(particle_positions, box_sizes)
                    
                # Compute mean displacement for the frame
                mean_displacement = compute_mean_displacement(unwrapped_positions)
                
                # Compute velocity for the frame
                velocity = compute_velocity(mean_displacement, time_interval)

                # Save velocity components along with phi
                results.append([phi, velocity[0], velocity[1], velocity[2]])

                print(f"For phi={phi}: Average velocity = {velocity}")
    
    # Sort results based on phi values
    results.sort(key=lambda x: x[0])
    
    # Write results to a text file
    with open("computational_velocity_FD.txt", "w") as outfile:
        outfile.write("Phi\tVelocity_X\tVelocity_Y\tVelocity_Z\n")
        for result in results:
            outfile.write(f"{result[0]}\t{result[1]}\t{result[2]}\t{result[3]}\n")
    
    print("Results saved to computational_velocity.txt")


Processing file: sed_N8000_phi0.30_FD.txt
For phi=0.3: Average velocity = [5.59577443e-03 6.78250004e-04 9.98251862e-01]
Processing file: sed_N8000_phi0.04_FD.txt
For phi=0.04: Average velocity = [-0.00698037 -0.00896906  0.99687341]
Processing file: sed_N8000_phi0.20_FD.txt
For phi=0.2: Average velocity = [ 9.92075942e-03 -5.14562644e-04  9.95895951e-01]
Processing file: sed_N8000_phi0.45_FD.txt
For phi=0.45: Average velocity = [-2.24941810e-03 -4.24097483e-04  1.00911720e+00]
Processing file: sed_N8000_phi0.02_FD.txt
For phi=0.02: Average velocity = [0.00943502 0.00558663 1.0019448 ]
Processing file: sed_N8000_phi0.10_FD.txt
For phi=0.1: Average velocity = [-0.00547921  0.00271387  0.99110637]
Processing file: sed_N8000_phi0.40_FD.txt
For phi=0.4: Average velocity = [ 2.01046931e-03 -9.14096869e-04  9.92695290e-01]
Processing file: sed_N8000_phi0.15_FD.txt
For phi=0.15: Average velocity = [0.00816601 0.00617516 0.99825492]
Processing file: sed_N8000_phi0.50_FD.txt
For phi=0.5: Averag